import pandas as pd
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line
# Assignment 2 (Part 3)

In [1]:
# Import Libraries

import json

from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import requests

In [2]:
# Load the data

Neigh_Loc_df = pd.read_csv('Part_2.csv')

<IPython.core.display.Javascript object>

In [3]:
# Extracting Boroughs that have cells that only contain Toronto

Toronto_Neighbourhoods = Neigh_Loc_df[Neigh_Loc_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [4]:
# Dropping the 'Unnamed: 0' column

Toronto_Neighbourhoods = Toronto_Neighbourhoods.drop(columns="Unnamed: 0")

In [5]:
# Dropping the 'Unnamed: 0.1' column

Toronto_Neighbourhoods = Toronto_Neighbourhoods.drop(columns="Unnamed: 0.1")

In [6]:
# Display the Dataframe

Toronto_Neighbourhoods

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [7]:
# Generate a map of Toronto using longitude and latitude values to visualize neighborhoods and how they cluster together

Latitude = Toronto_Neighbourhoods.loc[0, 'Latitude']
Longitude = Toronto_Neighbourhoods.loc[0, 'Longitude']

Map_of_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=13)

for Lati, Longi, Label in zip(Toronto_Neighbourhoods['Latitude'], Toronto_Neighbourhoods['Longitude'], Toronto_Neighbourhoods['Neighbourhood']):
    
    Label = folium.Popup(Label, parse_html=True)
    
    folium.CircleMarker(
        [Lati, Longi],
        radius=5,
        popup=Label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Map_of_Toronto)  
    
Map_of_Toronto

In [8]:
# Foursquare Credentials

Client_ID = '2ISUVAOPVA3IGPLHLHPUUGFODTENXS3PKWO4XE5FNLVUM5AE'
Client_Secret = 'UX1TGC4WDYGCRWKHHHVUXTMTZIUMTBAENMIQGMX52J25M1DD'
API_Version = '20180605'

In [9]:
# Obtaining the Neighbourhood Name

Neigh_Lati = Toronto_Neighbourhoods.loc[0, 'Latitude']
Neigh_Longi = Toronto_Neighbourhoods.loc[0, 'Longitude']

Neigh_Name = Toronto_Neighbourhoods.loc[0, 'Neighbourhood']

print('The Latitude and Longitude of ', Neigh_Name, 'is ', Neigh_Lati, 'and ', Neigh_Longi, 'respectively')

The Latitude and Longitude of  The Beaches is  43.67635739999999 and  -79.2930312 respectively


In [10]:
# 100 venues within a radius of 500 metres in 'The Beaches' neighbourhood will be used

Limit = 100

Radius = 500

In [11]:
# Create and Display the URL link

URL_Link = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    Client_ID, 
    Client_Secret, 
    API_Version, 
    Neigh_Lati, 
    Neigh_Longi, 
    Radius, 
    Limit)

URL_Link

'https://api.foursquare.com/v2/venues/explore?&client_id=2ISUVAOPVA3IGPLHLHPUUGFODTENXS3PKWO4XE5FNLVUM5AE&client_secret=UX1TGC4WDYGCRWKHHHVUXTMTZIUMTBAENMIQGMX52J25M1DD&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [12]:
# Sending a GET request and examining the results

Get_Requests = requests.get(URL_Link).json()
Get_Requests

{'meta': {'code': 200, 'requestId': '5ebb99349388d7001cec3581'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [13]:
# Function from the Foursquare Lab that extracts the category of the venue

def Category_Type(row):
    
    try:
        venue_categories = row['categories']
    except:
        venue_categories = row['venue.categories']
        
    if len(venue_categories) == 0:
        return None
    else:
        return venue_categories[0]['name']

In [14]:
# Cleaning and Structuring JSON into a pandas Dataframe

Venues = Get_Requests['response']['groups'][0]['items']
    
Venues_Nearby = json_normalize(Venues)

Filtered_Columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

Venues_Nearby = Venues_Nearby.loc[:, Filtered_Columns]

Venues_Nearby['venue.categories'] = Venues_Nearby.apply(Category_Type, axis=1)

Venues_Nearby.columns = [col.split(".")[-1] for col in Venues_Nearby.columns]

In [15]:
# Display the nearby venues

Venues_Nearby.head(10)

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [16]:
def NearbyVenues(Names, Latitudes, Longitudes, Radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(Names, Latitudes, Longitudes):
        print(name)
                    
        URL_Link = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
           Client_ID, 
           Client_Secret, 
           API_Version, 
           lat, 
           lng, 
           Radius, 
           Limit)
                   
        Get_Request = requests.get(URL_Link).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in Get_Request])

    Nearby_Venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    Nearby_Venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(Nearby_Venues)

Venues_in_Toronto = NearbyVenues(Names = Toronto_Neighbourhoods['Neighbourhood'],
                                 Latitudes = Toronto_Neighbourhoods['Latitude'],
                                 Longitudes = Toronto_Neighbourhoods['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

<IPython.core.display.Javascript object>

In [17]:
# Display the venues 

Venues_in_Toronto.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
5,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
8,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
9,"The Danforth West, Riverdale",43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio


In [18]:
# Analyzing each Neighbourhood

one_hot_encoding = pd.get_dummies(Venues_in_Toronto[['Venue Category']], prefix="", prefix_sep="")

one_hot_encoding['Neighbourhood'] = Venues_in_Toronto['Neighbourhood'] 

Fixed_Columns = [one_hot_encoding.columns[-1]] + list(one_hot_encoding.columns[:-1])

one_hot_encoding = one_hot_encoding[Fixed_Columns]

<IPython.core.display.Javascript object>

In [19]:
# Display the Venues 

one_hot_encoding.head(10)

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
# Grouping rows by their neighbourhood by calculating the mean of the frequency of occurrence of each category

Grouped_Rows = one_hot_encoding.groupby('Neighbourhood').mean().reset_index()

In [21]:
# Display the grouped Categories and Neighbourhoods

Grouped_Rows.head(10)

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
2,Business reply mail Processing Centre,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
4,Central Bay Street,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.014706,0.0,0.0,0.00,0.0,0.014706
5,Christie,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
6,Church and Wellesley,0.0,0.013699,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.013699,...,0.013699,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.027397
7,"Commerce Court, Victoria Hotel",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.040000,...,0.000000,0.000000,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.000000
8,Davisville,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.026316,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
9,Davisville North,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


In [22]:
# Printing each neighbourhood with the 5 most common venues

Most_Common_Venues = 5

for Hoods in Grouped_Rows['Neighbourhood']:
    
    print("+-+-"+Hoods+"+-+-")
    
    MCV = Grouped_Rows[Grouped_Rows['Neighbourhood'] == Hoods].T.reset_index()
    MCV.columns = ['venue','freq']
    MCV = MCV.iloc[1:]
    MCV['freq'] = MCV['freq'].astype(float)
    MCV = MCV.round({'freq': 2})
    
    print(MCV.sort_values('freq', ascending=False).reset_index(drop=True).head(Most_Common_Venues))
    
    print('\n')

+-+-Berczy Park+-+-
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3         Cheese Shop  0.04
4                Café  0.04


+-+-Brockton, Parkdale Village, Exhibition Place+-+-
                venue  freq
0                Café  0.13
1         Coffee Shop  0.09
2      Breakfast Spot  0.09
3                 Gym  0.04
4  Italian Restaurant  0.04


+-+-Business reply mail Processing Centre+-+-
                venue  freq
0         Yoga Studio  0.06
1       Auto Workshop  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


+-+-CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport+-+-
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2       Coffee Shop  0.06
3   Harbor / Marina  0.06
4  Sculpture Garden  0.06


+-+-Central Bay Street+-+-
                venue  freq
0         Coffee Shop  0.16
1              

In [23]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

Most_Common_Venues = 10

def most_common_venues(row, Most_Common_Venues):
    
    Categories_Row = row.iloc[1:]
    Categories_Row_Sorted = Categories_Row.sort_values(ascending=False)
    
    return Categories_Row_Sorted.index.values[0:Most_Common_Venues]

Indicator = ['st', 'nd', 'rd']

columns = ['Neighbourhood']

for Indi in np.arange(Most_Common_Venues):
    
    try:
        columns.append('{}{} Most Common Venue'.format(Indi+1, Indicator[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(Indi+1))

Sorted_Venues_in_Neighbourhoods = pd.DataFrame(columns = columns)

Sorted_Venues_in_Neighbourhoods['Neighbourhood'] = Grouped_Rows['Neighbourhood']

for Ind in np.arange(Grouped_Rows.shape[0]):
    
    Sorted_Venues_in_Neighbourhoods.iloc[Ind, 1:] = most_common_venues(Grouped_Rows.iloc[Ind, :], Most_Common_Venues)

Sorted_Venues_in_Neighbourhoods

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Beer Bar,Bakery,Café,Farmers Market,Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Bar,Burrito Place,Restaurant,Climbing Gym,Pet Store,Performing Arts Venue
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Sculpture Garden,Boutique,Rental Car Location,Coffee Shop,Boat or Ferry,Bar,Harbor / Marina,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Dessert Shop,Sandwich Place,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Thai Restaurant,Bar
5,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports,Coffee Shop
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gay Bar,Gastropub
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Japanese Restaurant
8,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Café,Gym,Italian Restaurant,Sushi Restaurant,Diner,Indian Restaurant
9,Davisville North,Park,Breakfast Spot,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Convenience Store,Gym,Doner Restaurant,Donut Shop


In [24]:
# Apply K-means clustering to cluster the neighbourhoods into 5 clusters

Clusters = 5

Grouped_Clusters = Grouped_Rows.drop('Neighbourhood', 1)

K_Means = KMeans(n_clusters=Clusters, random_state=0).fit(Grouped_Clusters)

K_Means.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
# Creating a Dataframe that will include the top 10 venues in each neighbourhood as well as all the clusters

Merged_Neighbourhoods = Toronto_Neighbourhoods

Merged_Neighbourhoods['Cluster Labels'] = K_Means.labels_

Merged_Neighbourhoods_merged = Merged_Neighbourhoods.join(Sorted_Venues_in_Neighbourhoods.set_index('Neighbourhood'), on='Neighbourhood')

In [26]:
# Display the above Dataframe

Merged_Neighbourhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0


In [27]:
# Visualizing the clusters by creating a map of the clusters

Mapping_Clusters = folium.Map(location=[Latitude, Longitude], zoom_start=13)

x = np.arange(Clusters)

y = [i+x+(i*x)**2 for i in range(Clusters)]

Color_Array = cm.rainbow(np.linspace(0, 1, len(y)))

Rainbow_Colors = [colors.rgb2hex(i) for i in Color_Array]

Markers_Colors = []

for Lati, Longi, Neigh, Cluster in zip(Merged_Neighbourhoods['Latitude'], Merged_Neighbourhoods['Longitude'], Merged_Neighbourhoods['Neighbourhood'], Merged_Neighbourhoods['Cluster Labels']):
    label = folium.Popup(str(Neigh) + ' Cluster ' + str(Cluster), parse_html=True)
    folium.CircleMarker(
        
        [Lati, Longi],
        radius = 5,
        popup = label,
        color = Rainbow_Colors[Cluster-1],
        fill = True,
        fill_color = Rainbow_Colors[Cluster-1],
        fill_opacity = 0.7).add_to(Mapping_Clusters)
       
Mapping_Clusters

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>